# Train model BK-AI

In [ ]:
! pip install -U sentence-transformers
! pip install -q wandb py_vncorenlp

import py_vncorenlp
py_vncorenlp.download_model(save_dir='/kaggle/working/')
import os
os.environ['JAVA_HOME'] = '/usr/lib/jvm/default-java'
rdrsegmenter = py_vncorenlp.VnCoreNLP(annotators=["wseg"], save_dir='/kaggle/working')

In [2]:
import pandas as pd
import numpy as np
import torch
import pickle
import math
import json
from datasets import load_dataset, DatasetDict, Dataset
from torch.utils.data import DataLoader
from sentence_transformers import SentenceTransformer, models, losses, util, datasets
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
from sentence_transformers.readers import InputExample
from sentence_transformers.cross_encoder.evaluation import CESoftmaxAccuracyEvaluator
from transformers import (
    AutoTokenizer,
    AutoConfig, 
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer)
import numpy as np

2024-06-12 02:21:59.961336: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-06-12 02:21:59.961452: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-06-12 02:22:00.085677: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
file_path = "/kaggle/input/law-train-evalute/mydata_trainset.json"
dataset = []
with open(file_path, "r") as file:
    for line in file:
        data = json.loads(line)
        dataset.append(data)

In [4]:
train_data = dataset[0]
train_data[0]

{'query': 'Các chi phí của tổng mức đầu tư xây dựng được quy định như thế nào?',
 'positive': 'Các chi phí của tổng mức đầu tư xây dựng được quy định tại Khoản 4 Điều 4 Nghị định 32/2015/NĐ-CP về Quản lý chi phí đầu tư xây dựng, cụ thể như sau: - Chi phí bồi thường, hỗ trợ và tái định cư gồm chi phí bồi thường về đất, nhà, công trình trên đất, các tài sản gắn liền với đất, trên mặt nước và chi phí bồi thường khác theo quy định; các khoản hỗ trợ khi nhà nước thu hồi đất; chi phí tái định cư; chi phí tổ chức bồi thường, hỗ trợ và tái định cư; chi phí sử dụng đất trong thời gian xây dựng (nếu có); chi phí chi trả cho phần hạ tầng kỹ thuật đã được đầu tư xây dựng (nếu có) và các chi phí có liên quan khác; - Chi phí xây dựng gồm chi phí phá dỡ các công trình xây dựng, chi phí san lấp mặt bằng xây dựng, chi phí xây dựng các công trình, hạng mục công trình, xây dựng công trình tạm, công trình phụ trợ phục vụ thi công; - Chi phí thiết bị gồm chi phí mua sắm thiết bị công trình và thiết bị công

In [5]:
import random

for _ in range(5):
    random.shuffle(train_data)

In [6]:
train_data = train_data[:80000]
# train_data = train_data[:20000]

In [7]:
segmented_train_data = []
for doc in train_data:
    segmented_train_data.append({
        'query' : ' '.join(rdrsegmenter.word_segment(doc['query'])),
        'positive' : ' '.join(rdrsegmenter.word_segment(doc['positive']))
    })

In [8]:
segmented_train_data[0]

{'query': 'Giao xe cho người không có bằng lái_xe sẽ bị phạt bao_nhiêu ?',
 'positive': 'Theo khoản 5 Điều 30 Nghị_định 100/2019 / NĐ-CP được sửa_đổi , bổ_sung bởi khoản 17 Điều 2 Nghị_định 123/2021/NĐ-CP quy_định về xử_phạt chủ phương_tiện vi_phạm quy_định liên_quan đến giao_thông đường_bộ như sau : đ ) Giao xe hoặc để cho người không đủ điều_kiện theo quy_định tại khoản 1 Điều 58 của Luật Giao_thông đường_bộ điều_khiển xe tham_gia giao_thông ( bao_gồm cả trường_hợp người điều_khiển phương_tiện có Giấy_phép lái_xe nhưng đã hết hạn sử_dụng hoặc đang trong thời_gian bị tước quyền sử_dụng ) ; . 5 . Phạt tiền từ 800.000 đồng đến 2.000.000 đồng đối_với cá_nhân , từ 1.600.000 đồng đến 4.000.000 đồng đối_với tổ_chức là chủ xe mô_tô , xe_gắn_máy và các loại xe tương_tự xe mô_tô thực_hiện một trong các hành_vi vi_phạm sau đây : d ) Khai_báo không đúng sự_thật hoặc sử_dụng các giấy_tờ , tài_liệu giả để được cấp lại biển số , Giấy đăng_ký xe ; e ) Không chấp_hành việc thu_hồi Giấy đăng_ký xe , b

In [9]:
train_set = []
train_set = [InputExample(texts = [a['query'], a['positive']]) for a in train_data]

In [10]:
word_embedding_model = models.Transformer("bkai-foundation-models/vietnamese-bi-encoder", max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(), pooling_mode = 'mean')

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

config.json:   0%|          | 0.00/777 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/540M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/895k [00:00<?, ?B/s]

bpe.codes:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/167 [00:00<?, ?B/s]

In [11]:
num_epochs = 5
train_dataloader = DataLoader(train_set, shuffle=True, batch_size=32)
train_loss = losses.MultipleNegativesRankingLoss(model=model)
warmup_steps = math.ceil(len(train_dataloader) * num_epochs  * 0.1)

In [12]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=num_epochs,
    warmup_steps=warmup_steps,
    optimizer_class=torch.optim.AdamW,
    optimizer_params={
        'lr': 2e-5
    },
    weight_decay=0.01,
    output_path="/kaggle/working/finetuned_bkai_5epoch_80k"
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss
500,0.112100
1000,0.072800
1500,0.059700
2000,0.053100
2500,0.044700
3000,0.035900
3500,0.022100
4000,0.012500
4500,0.011500
5000,0.009700


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

# Evaluate

In [3]:
import json
import pickle

with open('/kaggle/input/law-train-evalute/chunk_id_mapping', 'rb') as f:
    idx_mapping = pickle.load(f)
with open('/kaggle/input/law-train-evalute/chunk_corpus', 'rb') as f:
    segmented_chunk_corpus = pickle.load(f)
with open('/kaggle/input/law-train-evalute/queries_relevant_chunks', 'rb') as f:
    segmented_queries_relevant_chunk = pickle.load(f)
    
    
with open('/kaggle/input/law-qa/query_set_evaluate.json', 'r') as file:
    query_test_set = [json.loads(line) for line in file]

In [4]:
print(segmented_chunk_corpus[10])

{'doc_id': '32/2024/qh15_4', 'chunk_id': '32/2024/qh15_4_3', 'title': 'Điều 4 .Giải_thích từ_ngữ', 'chunk': '3 . Can_thiệp sớm là việc Ngân_hàng Nhà_nước Việt_Nam ( sau đây gọi là Ngân_hàng Nhà_nước ) áp_dụng các yêu_cầu , biện_pháp hạn_chế đối_với tổ_chức tín_dụng , chi_nhánh ngân_hàng nước_ngoài và yêu_cầu tổ_chức tín_dụng , chi_nhánh ngân_hàng nước_ngoài đó thực_hiện phương_án khắc_phục dưới sự giám_sát của Ngân_hàng Nhà_nước nhằm khắc_phục tình_trạng theo quy_định tại khoản 1 Điều 156 của Luật này .', 'text': 'Điều 4 .Giải_thích từ_ngữ :3 . Can_thiệp sớm là việc Ngân_hàng Nhà_nước Việt_Nam ( sau đây gọi là Ngân_hàng Nhà_nước ) áp_dụng các yêu_cầu , biện_pháp hạn_chế đối_với tổ_chức tín_dụng , chi_nhánh ngân_hàng nước_ngoài và yêu_cầu tổ_chức tín_dụng , chi_nhánh ngân_hàng nước_ngoài đó thực_hiện phương_án khắc_phục dưới sự giám_sát của Ngân_hàng Nhà_nước nhằm khắc_phục tình_trạng theo quy_định tại khoản 1 Điều 156 của Luật này .'}


In [5]:
print(query_test_set[10])

{'query_id': '66635835c2f544363ef20db0', 'query': '03 độ mật của bí mật nhà nước là gì?', 'relevant_docs': ['29/2018/qh14_8']}


In [6]:
query_test_relevant_chunks = []

for query in query_test_set:
    
    relevant_chunks = []
    
    for relevant_doc in query['relevant_docs']:
        try:
            relevant_chunks += idx_mapping[relevant_doc]
        except KeyError:
            continue
        
    query_test_relevant_chunks.append({
        'query_id' : query['query_id'],
        'query' : query['query'],
        'relevant_docs' : query['relevant_docs'],
        'relevant_chunks' : relevant_chunks
    })

In [11]:
for q in query_test_relevant_chunks:
    if q['query_id'] == '66635708c2f544363eed5946':
        print(q)

{'query_id': '66635708c2f544363eed5946', 'query': 'Ai có thẩm quyền chủ trì cuộc họp Hội đồng thành viên doanh nghiệp nhà nước?', 'relevant_docs': ['59/2020/qh14_98_2'], 'relevant_chunks': []}


In [32]:
segmented_query_test_relevant_chunks = []

for query in query_test_relevant_chunks:
    segmented_query_test_relevant_chunks.append({
        'query_id' : query['query_id'],
        'query' : ''.join(rdrsegmenter.word_segment(query['query'])),
        'relevant_docs' : query['relevant_docs'],
        'relevant_chunks' : query['relevant_chunks']
    })

In [33]:
from sentence_transformers.evaluation import InformationRetrievalEvaluator

corpus = {}
queries = {}
relevant_docs = {}
corpus_idx = 0
query_idx = 0

for query in segmented_query_test_relevant_chunks:
    queries[query['query_id']] = query['query']
    relevant_docs[query['query_id']] = set(query['relevant_chunks'])

for doc in segmented_chunk_corpus:
    corpus[doc['chunk_id']] = doc['text']

In [19]:
# import random
# shuffled_query_ids = list(queries.keys())
# random.shuffle(shuffled_query_ids)

# eval_query_ids = shuffled_query_ids[:4000]

# queries = {query_id: queries[query_id] for query_id in eval_query_ids}

## Evalute 20k - 5 epochs

In [20]:
IR_evaluator = InformationRetrievalEvaluator(queries,corpus,relevant_docs)
model.evaluate(IR_evaluator, output_path="/kaggle/working/finetuned_bkai_5epoch_20k_chunk_without_semicolon")

{'cosine_accuracy@1': 0.5445884146341463,
 'cosine_accuracy@3': 0.7496189024390244,
 'cosine_accuracy@5': 0.8003048780487805,
 'cosine_accuracy@10': 0.8597560975609756,
 'cosine_precision@1': 0.5445884146341463,
 'cosine_precision@3': 0.4447408536585366,
 'cosine_precision@5': 0.35769817073170734,
 'cosine_precision@10': 0.2331173780487805,
 'cosine_recall@1': 0.18538996561524934,
 'cosine_recall@3': 0.42904631578617075,
 'cosine_recall@5': 0.5475906496107235,
 'cosine_recall@10': 0.6855533470581276,
 'cosine_ndcg@10': 0.5959769122051308,
 'cosine_mrr@10': 0.6574301926054971,
 'cosine_map@100': 0.5302161117578659,
 'dot_accuracy@1': 0.5350609756097561,
 'dot_accuracy@3': 0.7336128048780488,
 'dot_accuracy@5': 0.7976371951219512,
 'dot_accuracy@10': 0.8567073170731707,
 'dot_precision@1': 0.5350609756097561,
 'dot_precision@3': 0.43241869918699183,
 'dot_precision@5': 0.3512957317073171,
 'dot_precision@10': 0.2291158536585366,
 'dot_recall@1': 0.18155795683065357,
 'dot_recall@3': 0.41

In [21]:
import pandas as pd
df = pd.read_csv('/kaggle/working/finetuned_bkai_5epoch_20k_chunk_without_semicolon/Information-Retrieval_evaluation_results.csv')
df

,epoch,steps,cosine-Accuracy@1,cosine-Accuracy@3,cosine-Accuracy@5,cosine-Accuracy@10,cosine-Precision@1,cosine-Recall@1,cosine-Precision@3,cosine-Recall@3,...,dot-Recall@1,dot-Precision@3,dot-Recall@3,dot-Precision@5,dot-Recall@5,dot-Precision@10,dot-Recall@10,dot-MRR@10,dot-NDCG@10,dot-MAP@100
0,-1,-1,0.544588,0.749619,0.800305,0.859756,0.544588,0.18539,0.444741,0.429046,...,0.181558,0.432419,0.415842,0.351296,0.537485,0.229116,0.674013,0.649267,0.584678,0.518315


## Evalute 80k - 5 epochs

In [34]:
IR_evaluator = InformationRetrievalEvaluator(queries,corpus,relevant_docs)
model.evaluate(IR_evaluator, output_path="/kaggle/working/finetuned_bkai_5epoch_80k_chunk_without_semicolon")

{'cosine_accuracy@1': 0.5466932535726154,
 'cosine_accuracy@3': 0.7480890661349285,
 'cosine_accuracy@5': 0.8068295114656032,
 'cosine_accuracy@10': 0.8668162180126288,
 'cosine_precision@1': 0.5466932535726154,
 'cosine_precision@3': 0.4366622355156751,
 'cosine_precision@5': 0.3553173811897641,
 'cosine_precision@10': 0.23216184778996346,
 'cosine_recall@1': 0.18576450464684233,
 'cosine_recall@3': 0.4181245222395017,
 'cosine_recall@5': 0.542569408260353,
 'cosine_recall@10': 0.6760155016976835,
 'cosine_ndcg@10': 0.5901858246453324,
 'cosine_mrr@10': 0.6599634957561126,
 'cosine_map@100': 0.5225830129741482,
 'dot_accuracy@1': 0.5314057826520439,
 'dot_accuracy@3': 0.7406945829179129,
 'dot_accuracy@5': 0.7997673645729478,
 'dot_accuracy@10': 0.8602525756065138,
 'dot_precision@1': 0.5314057826520439,
 'dot_precision@3': 0.42746759720837485,
 'dot_precision@5': 0.34669325357261543,
 'dot_precision@10': 0.22835659687603854,
 'dot_recall@1': 0.1796938147158583,
 'dot_recall@3': 0.408

In [35]:
import pandas as pd
df = pd.read_csv('/kaggle/working/finetuned_bkai_5epoch_20k_chunk_without_semicolon/Information-Retrieval_evaluation_results.csv')
df

,epoch,steps,cosine-Accuracy@1,cosine-Accuracy@3,cosine-Accuracy@5,cosine-Accuracy@10,cosine-Precision@1,cosine-Recall@1,cosine-Precision@3,cosine-Recall@3,...,dot-Recall@1,dot-Precision@3,dot-Recall@3,dot-Precision@5,dot-Recall@5,dot-Precision@10,dot-Recall@10,dot-MRR@10,dot-NDCG@10,dot-MAP@100
0,-1,-1,0.546693,0.748089,0.80683,0.866816,0.546693,0.185765,0.436662,0.418125,...,0.179694,0.427468,0.408142,0.346693,0.528401,0.228357,0.66511,0.647828,0.57783,0.509889
1,-1,-1,0.546693,0.748089,0.80683,0.866816,0.546693,0.185765,0.436662,0.418125,...,0.179694,0.427468,0.408142,0.346693,0.528401,0.228357,0.66511,0.647828,0.57783,0.509889
